In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 1.0)
cnt = resample_cnt(cnt, 2.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

inputs_a = np_to_var(x_right[:160,0:1,:,None], dtype=np.float32).cuda()

inputs_b = np_to_var(x_rest[:160,0:1,:,None], dtype=np.float32).cuda()
inputs = [inputs_a, inputs_b]

In [ ]:
fig = plt.figure(figsize=(8,4))
for i_class in range(2):
    ins = var_to_np(inputs[i_class].squeeze())
    bps = np.abs(np.fft.rfft(ins.squeeze()))
    plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

    
plt.title("Spectrum")
plt.xlabel('Frequency [Hz]')

plt.ylabel('Amplitude')
plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
display(fig)
plt.close(fig)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(x_right[:,0], x_right[:,1], ls='', marker='o')

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c,),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2))
    )
    
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Linear(n_c, n_i_c),
        nn.ReLU(),
            nn.Linear(n_i_c, n_c)),
    )

In [ ]:
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    rev_block(2,50),
    rev_block(2,50),
    #rev_block(2,50),
)
feature_model.cuda()


In [ ]:


from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    res_block(2,100),
    res_block(2,100),
    )


adv_model = ProjectionDiscriminator(adv_model,2,2,)

adv_model.cuda()

In [ ]:
from reversible.training import hard_init_std_mean
n_dims = inputs_b.shape[2]
n_clusters = 2
means_per_cluster = [th.autograd.Variable(th.ones(n_dims).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                    for _ in range(n_clusters)]


for i_class in range(2):
    this_outs = feature_model(inputs[i_class].squeeze())
    means_per_cluster[i_class].data = th.mean(this_outs, dim=0).data
    stds_per_cluster[i_class].data = th.log(th.std(this_outs, dim=0)).data

In [ ]:
from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))


optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))

In [ ]:
from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan

In [ ]:
n_epochs = 1001
i_class_dim = 1
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_adv.zero_grad()
    optimizer.zero_grad()
    optim_dist.zero_grad()
    for i_class in range(len(inputs)):
        mean = means_per_cluster[i_class]
        log_std = stds_per_cluster[i_class]
        if i_class == 1:
            mean = th.cat((means_per_cluster[0][0:i_class_dim], mean[i_class_dim:i_class_dim+1],
                         means_per_cluster[0][i_class_dim+1:]))
            log_std = th.cat((stds_per_cluster[0][0:i_class_dim], log_std[i_class_dim:i_class_dim+1],
                         stds_per_cluster[0][i_class_dim+1:]))
        std = th.exp(log_std)
        this_inputs = inputs[i_class]
        y = np_to_var([i_class]).cuda()
        samples = get_gauss_samples(len(this_inputs), mean, std, truncate_to=3)
        other_outs = feature_model(inputs[i_class - 1].squeeze())
        
        # move to this class
        other_outs[:,i_class_dim] = means_per_cluster[i_class][i_class_dim].unsqueeze(0) + (
            th.randn(len(other_outs)).cuda() * stds_per_cluster[i_class][i_class_dim])
        samples = th.cat((samples, other_outs), dim=0)

        inverted = invert(feature_model, samples)
        score_fake = adv_model(inverted, y)
        if (i_epoch % 10) != 0:
            score_real = adv_model(this_inputs.squeeze(), y)
            gradient_loss = gradient_penalty(adv_model, this_inputs.squeeze(), inverted[:(len(this_inputs))], y)
            d_loss = -score_real.mean() + score_fake.mean() + gradient_loss * 10
            d_loss.backward()
            d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
        else:
            g_loss = -th.mean(score_fake)
            g_loss.backward()
            g_grad = np.mean([th.sum(p.grad **2).item() for p in feature_model.parameters()])
    if (i_epoch % 10) != 0:
            optim_adv.step()
    else:
        optimizer.step()
        optim_dist.step()
    

    with th.no_grad():
        sample_wd_row = {}
        for i_class in range(len(inputs)):
            this_inputs = inputs[i_class]
            mean = means_per_cluster[i_class]
            log_std = stds_per_cluster[i_class]
            if i_class == 1:
                mean = th.cat((means_per_cluster[0][0:i_class_dim], mean[i_class_dim:i_class_dim+1],
                             means_per_cluster[0][i_class_dim+1:]))
                log_std = th.cat((stds_per_cluster[0][0:i_class_dim], log_std[i_class_dim:i_class_dim+1],
                             stds_per_cluster[0][i_class_dim+1:]))
            std = th.exp(log_std)
            y = np_to_var([i_class]).cuda()
            samples = get_gauss_samples(len(this_inputs), mean, std, truncate_to=3)

            inverted = invert(feature_model, samples)
            in_np = var_to_np(this_inputs).reshape(len(this_inputs), -1)
            fake_np = var_to_np(inverted).reshape(len(inverted), -1)
            import ot

            dist = np.sqrt(np.sum(np.square(in_np[:,None] - fake_np[None]), axis=2))
            match_matrix = ot.emd([],[], dist)
            cost = np.sum(dist * match_matrix)
            score_fake = adv_model(inverted, y)
            score_real = adv_model(inputs[i_class].squeeze(), y)
            wd_dist = th.mean(score_real) - th.mean(score_fake)
            sample_wd_row.update({
                'wd_dist_' + str(i_class): wd_dist.item() ,
                'sampled_wd' + str(i_class): cost,
                'wd_diff' + str(i_class): cost - wd_dist.item(),
            })
        end_time = time.time()
        epoch_row = {
        'd_loss': d_loss.item(),
        'g_loss': g_loss.item(),
        'o_real': th.mean(score_real).item(),
        'o_fake': th.mean(score_fake).item(),
        'g_grad': g_grad,
        'd_grad': d_grad,
        'runtime': end_time -start_time,}
        epoch_row.update(sample_wd_row)
        df = df.append(epoch_row, ignore_index=True)
        if i_epoch % (max(1,n_epochs // 100)) == 0:
            display_text("Epoch {:d}".format(i_epoch))
            display(df.iloc[-5:])

        if i_epoch % (n_epochs // 20) == 0:
            print("stds\n", var_to_np(th.exp(th.stack(stds_per_cluster))))
            for i_class in range(len(stds_per_cluster)):
                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                if i_class == 1:
                    mean = th.cat((means_per_cluster[0][0:i_class_dim], mean[i_class_dim:i_class_dim+1],
                                 means_per_cluster[0][i_class_dim+1:]))
                    log_std = th.cat((stds_per_cluster[0][0:i_class_dim], log_std[i_class_dim:i_class_dim+1],
                                 stds_per_cluster[0][i_class_dim+1:]))
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                samples = get_gauss_samples(len(this_inputs), mean, std, truncate_to=3)
                inverted = invert(feature_model, samples).squeeze()
                inverted = var_to_np(inverted)
                fig, axes = plt.subplots(10,8,
                    sharex=True, sharey=True,
                    figsize=(int(10), int(8)))
                plt.subplots_adjust(wspace=0, hspace=0)
                for ax, curve  in zip(axes.flatten(), inverted.reshape(-1, inverted.shape[-1])):
                    ax.plot(curve, color=seaborn.color_palette()[i_class])
                display(fig)
                plt.close(fig)

            fig = plt.figure(figsize=(8,4))
            for i_class in range(2):
                ins = var_to_np(inputs[i_class].squeeze())
                bps = np.abs(np.fft.rfft(ins.squeeze()))
                plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                if i_class == 1:
                    mean = th.cat((means_per_cluster[0][0:i_class_dim], mean[i_class_dim:i_class_dim+1],
                                 means_per_cluster[0][i_class_dim+1:]))
                    log_std = th.cat((stds_per_cluster[0][0:i_class_dim], log_std[i_class_dim:i_class_dim+1],
                                 stds_per_cluster[0][i_class_dim+1:]))
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                samples = get_gauss_samples(5000, mean, std, truncate_to=3)
                inverted = var_to_np(invert(feature_model, samples).squeeze())
                bps = np.abs(np.fft.rfft(inverted.squeeze()))
                plt.plot(np.fft.rfftfreq(inverted.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0),
                        color=seaborn.color_palette()[i_class], ls='--')
            plt.title("Spectrum")
            plt.xlabel('Frequency [Hz]')

            plt.ylabel('Amplitude')
            plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
            display(fig)
            plt.close(fig)
            
            for i_class in range(2):
                fig = plt.figure(figsize=(5,5))
                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                if i_class == 1:
                    mean = th.cat((means_per_cluster[0][0:i_class_dim], mean[i_class_dim:i_class_dim+1],
                                 means_per_cluster[0][i_class_dim+1:]))
                    log_std = th.cat((stds_per_cluster[0][0:i_class_dim], log_std[i_class_dim:i_class_dim+1],
                                 stds_per_cluster[0][i_class_dim+1:]))
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                samples = get_gauss_samples(5000, mean, std, truncate_to=3)
                inverted = var_to_np(invert(feature_model, samples).squeeze())
                plt.plot(inverted.squeeze()[:,0], inverted.squeeze()[:,1],
                         ls='', marker='o', color=seaborn.color_palette()[i_class + 2], alpha=0.5, markersize=2)
                plt.plot(var_to_np(inputs[i_class].squeeze())[:,0], var_to_np(inputs[i_class].squeeze())[:,1],
                         ls='', marker='o', color=seaborn.color_palette()[i_class])

                display(fig)
                plt.close(fig)

In [ ]:
 for i_class in range(2):
    this_ins = inputs[i_class].squeeze()
    this_outs = feature_model(this_ins.squeeze())
    fig = plt.figure(figsize=(5,5))
    plt.plot(var_to_np(this_ins)[:,0], var_to_np(this_ins)[:,1],
             ls='', marker='o', color=seaborn.color_palette()[i_class])
    plt.plot(var_to_np(this_outs).squeeze()[:,0], var_to_np(this_outs).squeeze()[:,1],
             ls='', marker='o', color=seaborn.color_palette()[i_class + 2])

    cur_x = inputs[i_class].squeeze()
    all_outs = [cur_x]
    for m in feature_model.children():
        cur_x = m(cur_x)
        all_outs.append(cur_x)
    for one_in, one_out in zip(var_to_np(this_ins), var_to_np(this_outs)):
        comb = np.stack((one_in, one_out))
        plt.plot(comb[:,0], comb[:,1], lw=0.25, color='black')
    display(fig)
    plt.close(fig)
    fig = plt.figure(figsize=(5,5))
    cmap = LinearSegmentedColormap.from_list('twoclass', [seaborn.color_palette()[i_class],
                                    seaborn.color_palette()[i_class+2]],
                              N=len(all_outs))

    colors = cmap(np.linspace(0,1, len(all_outs)))
    for i_l, (m_in, m_out) in enumerate(zip(all_outs[:-1], all_outs[1:])):
        plt.plot(var_to_np(m_in)[:,(i_l % 2)], var_to_np(m_in)[:,1-(i_l % 2)],
             ls='', marker='o', color=colors[i_l])
        plt.plot(var_to_np(m_out).squeeze()[:,1-(i_l % 2)], var_to_np(m_out).squeeze()[:,(i_l % 2)],
                 ls='', marker='o', color=colors[i_l+1])
        for one_in, one_out in zip(var_to_np(m_in), var_to_np(m_out)):
            comb = np.stack((one_in, one_out))
            if i_l % 2 == 0:
                plt.plot([one_in[0], one_out[1]], [one_in[1], one_out[0]],
                        lw=0.25, color='black')
            else:
                plt.plot([one_in[1], one_out[0]], [one_in[0], one_out[1]],
                        lw=0.25, color='black')
    display(fig)
    plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(5,5))
colors = cmap(np.linspace(0,1, len(all_outs)))
for i_l, (m_in, m_out) in enumerate(zip(all_outs[:-1], all_outs[1:])):
    plt.plot(var_to_np(m_in)[:,0], var_to_np(m_in)[:,1],
         ls='', marker='o', color=colors[i_l])
    plt.plot(var_to_np(m_out).squeeze()[:,0], var_to_np(m_out).squeeze()[:,1],
             ls='', marker='o', color=colors[i_l+1])
    for one_in, one_out in zip(var_to_np(m_in), var_to_np(m_out)):
        comb = np.stack((one_in, one_out))
        plt.plot(comb[:,0], comb[:,1], lw=0.25, color='black')
display(fig)
plt.close(fig)

In [ ]:

fig = plt.figure(figsize=(8,4))
for i_class in range(2):
    ins = var_to_np(inputs[i_class].squeeze())
    bps = np.abs(np.fft.rfft(ins.squeeze()))
    plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.mean(bps, axis=0))

    mean = means_per_cluster[i_class]
    log_std = stds_per_cluster[i_class]
    if i_class == 1:
        mean = th.cat((means_per_cluster[0][0:i_class_dim], mean[i_class_dim:i_class_dim+1],
                     means_per_cluster[0][i_class_dim+1:]))
        log_std = th.cat((stds_per_cluster[0][0:i_class_dim], log_std[i_class_dim:i_class_dim+1],
                     stds_per_cluster[0][i_class_dim+1:]))
    std = th.exp(log_std)
    y = np_to_var([i_class]).cuda()
    samples = get_gauss_samples(5000, mean, std, truncate_to=3)
    inverted = var_to_np(invert(feature_model, samples).squeeze())
    bps = np.abs(np.fft.rfft(inverted.squeeze()))
    plt.plot(np.fft.rfftfreq(inverted.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.mean(bps, axis=0),
            color=seaborn.color_palette()[i_class], ls='--')
plt.title("Spectrum")
plt.xlabel('Frequency [Hz]')

plt.ylabel('Amplitude')
plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
display(fig)
plt.close(fig)

In [ ]:
comb.shape

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
 for i_class in range(2):
    this_ins = inputs[i_class].squeeze()
    this_outs = feature_model(this_ins.squeeze())
    fig = plt.figure(figsize=(5,5))
    plt.plot(var_to_np(this_ins)[:,0], var_to_np(this_ins)[:,1],
             ls='', marker='o', color=seaborn.color_palette()[i_class])
    plt.plot(var_to_np(this_outs).squeeze()[:,0], var_to_np(this_outs).squeeze()[:,1],
             ls='', marker='o', color=seaborn.color_palette()[i_class + 2])

    cur_x = inputs[i_class].squeeze()
    all_outs = [cur_x]
    for m in feature_model.children():
        cur_x = m(cur_x)
        all_outs.append(cur_x)
    for one_in, one_out in zip(var_to_np(this_ins), var_to_np(this_outs)):
        comb = np.stack((one_in, one_out))
        plt.plot(comb[:,0], comb[:,1], lw=0.25, color='black')
    display(fig)
    plt.close(fig)
    #continue
    fig = plt.figure(figsize=(5,5))
    cmap = LinearSegmentedColormap.from_list('twoclass', [seaborn.color_palette()[i_class],
                                    seaborn.color_palette()[i_class+2]],
                              N=len(all_outs))

    colors = cmap(np.linspace(0,1, len(all_outs)))
    for i_l, (m_in, m_out) in enumerate(zip(all_outs[:-1], all_outs[1:])):
        plt.plot(var_to_np(m_in)[:,0], var_to_np(m_in)[:,1],
             ls='', marker='o', color=colors[i_l])
        plt.plot(var_to_np(m_out).squeeze()[:,0], var_to_np(m_out).squeeze()[:,1],
                 ls='', marker='o', color=colors[i_l+1])
        for one_in, one_out in zip(var_to_np(m_in), var_to_np(m_out)):
            comb = np.stack((one_in, one_out))
            plt.plot(comb[:,0], comb[:,1], lw=0.25, color='black')
    display(fig)
    plt.close(fig)

In [ ]:
 for i_class in range(2):
    fig = plt.figure(figsize=(5,5))
    this_ins = inputs[i_class].squeeze()
    this_outs = feature_model(this_ins.squeeze())
    plt.plot(var_to_np(this_outs).squeeze()[:,0], var_to_np(this_outs).squeeze()[:,1],
             ls='', marker='o', color=seaborn.color_palette()[i_class + 2])
    plt.plot(var_to_np(this_ins)[:,0], var_to_np(this_ins)[:,1],
             ls='', marker='o', color=seaborn.color_palette()[i_class])

    
    display(fig)
    plt.close(fig)

In [ ]:
means_per_cluster

In [ ]:
i_class_dim

In [ ]:
stds_per_cluster